In [ ]:
!pip install beautifulsoup4

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import os
import io
%matplotlib inline

In [ ]:
def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root,filename)
            
            inBody = False
            lines = []
            
            f = io.open(path, 'r', encoding = 'latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message
                    

In [ ]:
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in read_files(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    return pd.DataFrame(rows, index=index)

In [ ]:
data = pd.DataFrame({'message':[],'class':[]})
data = data.append(dataFrameFromDirectory('/home/ian/Documents/ham-and-spam-dataset/ham',0))
data = data.append(dataFrameFromDirectory('/home/ian/Documents/ham-and-spam-dataset/spam',1))
data

In [ ]:
total_mails = data['message'].shape[0]
total_mails

In [ ]:
from bs4 import BeautifulSoup

def func(df):
    soup = BeautifulSoup(df['message'], "html.parser").find()
    if bool(soup):
#         is_html = soup.find('html')
#         df['html'] = 'True'
        
#         if bool(is_html):
#             df['mezz'] = 'True'
#         soup = BeautifulSoup(df['message'], "html.parser")
    
        text = soup.find_all(text=True)
        print(text)
        df['message'] = text
        
        return df['message']
    else:
        
        return df['message'] 

In [ ]:
from bs4 import BeautifulSoup


def generate_blacklist(df):
    blacklist = []
    soup = BeautifulSoup(df['message'], "html.parser")
    text = soup.find_all(text=True)
    blacklist+=set([t.parent.name for t in text])
    return blacklist

In [ ]:
def cleaning_html(df, blacklist):
    soup = BeautifulSoup(df['message'], "html.parser")
    
    text = soup.get_text()

#     output = ''

#     for t in text:
#         if t not in blacklist:
#             output += '{} '.format(t)
#     print(text)
    return text

In [ ]:
def checking_html(df):
    count = 0
    soup = BeautifulSoup(df['message'], "html.parser").find()
    if bool(soup):
        count+=1
    return count
    

In [ ]:
new_data = data.reset_index()
new_data

In [ ]:
train_index, test_index = list(), list()
for i in range(data['message'].shape[0]):
    if np.random.uniform(0,1) < 0.75:
        train_index.append(i)
    else:
        test_index.append(i)
        
train_data = new_data.loc[train_index]
test_data = new_data.loc[test_index]

In [ ]:
train_data.shape[0]

In [ ]:
train_data['message']= train_data.apply(func, axis=1)
train_data

In [ ]:
train_data.apply(checking_html, axis=1)

In [ ]:
black = train_data[train_data.html == 'True'].apply(generate_blacklist,axis = 1)
# flat_list = [item for sublist in l for item in sublist]
black_list = [item for sublist in black.values for item in sublist]
blacklist_set = set(black_list)

In [ ]:
blacklist_set

In [ ]:
train_with_html = train_data[train_data.html == 'True']
train_data['message'] = train_with_html.apply(cleaning_html, blacklist=black_list,axis = 1)
train_data

In [ ]:
black

In [ ]:
train_data.apply(func, axis=1)

In [ ]:
train_no_html = train_data[train_data.html == 'False']
train_no_html

In [ ]:
train_with_html = train_data[train_data.html == 'True']
train_with_html['message'] = train_with_html.apply(cleaning_html, blacklist=black_list,axis = 1)

In [ ]:
train_with_html

In [ ]:
train_with_html['html']= train_with_html.apply(func, axis=1)
train_data

In [ ]:
html_data = train_data[train_data['html'] == 'True']
html_data['message'].head(10)


In [ ]:
train_data.columns

In [ ]:
train_data[train_data['mezz']!='False']

In [ ]:
train_data.reset_index(inplace=True)
train_data.drop(['index'],axis =1, inplace=True)
train_data

In [ ]:
import BeautifulSoup4

bool(BeautifulSoup(train_data['message'], "html.parser").find())

In [ ]:
spam_words = ' '.join(list(train_data[train_data['class'] == 1]['message']))
spamwc = WordCloud(width = 512, height=512).generate(spam_words)
plt.figure(figsize=(10,8), facecolor='k')
plt.imshow(spamwc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
-